In [ ]:
# install.packages('redux')

In [1]:
library(ggplot2)
library(SingleR)
library(dplyr)
library(celldex)
library(RColorBrewer)
library(stringr)
library(DoubletFinder)

source("../tools/formating/formating.R")

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    rowOrderStats, rowProds, rowQuantiles, rowRanges

In [2]:
input="/ps/ai-ready/data/error/droplet_Bladder_seurat_tiss.h5seurat"
output='/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat'
adata_path = '/ps/ai-ready/data/error/droplet_Bladder_seurat_tiss.h5ad'
unique_id='Seurat test'
assay='RNA'
min_genes=200
max_genes=0
min_UMI_count=0
max_UMI_count=0
percent_mt_max=5
percent_rb_min=0
resolution=0.5
dims=1:10
doublet_rate=0.075
regress_cell_cycle=FALSE

In [3]:
srat <- tryCatch(
        LoadSeurat(input),
        error = function(e) {
            # stop("The file format is not supported.")
            # print(e)
            RedisError(unique_id, paste0("The file format is not supported: ", e$message))
            stop(paste0("The file format is not supported: ", e$message))
        }
    )

[1] "Inside LoadSeurat"


Validating h5Seurat file

Initializing RNA with data

Adding counts for RNA

Adding miscellaneous information for RNA

Adding command information

Adding cell-level metadata

Adding miscellaneous information

Adding tool-specific results



In [4]:
srat

An object of class Seurat 
23341 features across 2500 samples within 1 assay 
Active assay: RNA (23341 features, 0 variable features)
 2 layers present: counts, data

In [5]:
srat <- NormalizeData(srat, normalization.method = "LogNormalize", scale.factor = 10000)

In [6]:
srat <- FindVariableFeatures(srat, selection.method = "vst", nfeatures = 2000)

In [7]:
srat

An object of class Seurat 
23341 features across 2500 samples within 1 assay 
Active assay: RNA (23341 features, 2000 variable features)
 2 layers present: counts, data

In [9]:
VariableFeatures(srat)

[1] "Cd74"          "H2-Aa"         "H2-Ab1"        "H2-Eb1"       
   [5] "Lyz2"          "Lyz1"          "C1qa"          "C1qb"         
   [9] "Il1b"          "C1qc"          "Apoe"          "Cd14"         
  [13] "Cd83"          "Sprr2f"        "Cxcl2"         "Tyrobp"       
  [17] "Sprr2g"        "Rn45s"         "Fcer1g"        "Ctss"         
  [21] "Rgs1"          "Ccl7"          "Cxcl10"        "Srgn"         
  [25] "Ccl9"          "Psca"          "Cd52"          "Ccl2"         
  [29] "Aqp1"          "Pf4"           "Ccl4"          "Cytip"        
  [33] "Fam25c"        "Cxcl1"         "Gm11428"       "Bcl2a1b"      
  [37] "Ccl3"          "Il33"          "Acta2"         "Mgl2"         
  [41] "Bcl2a1d"       "Cd209a"        "Ctla2a"        "Plvap"        
  [45] "Sprr2b"        "Rgs5"          "Cldn5"         "H2-DMb1"      
  [49] "Bglap2"        "Igfbp3"        "Coro1a"        "Stmn2"        
  [53] "Ccl6"          "Rrad"          "Pecam1"        "Retnla"       
  [57] "Emcn"          "Plbd1"         "Pcp4l1"        "Tm4sf1"       
  [61] "Clec3b"        "Gm13889"       "H2-DMa"        "Clec10a"      
  [65] "Slpi"          "Xist"          "Cdh5"          "Plek"         
  [69] "Vcam1"         "Egfl7"         "Slfn2"         "Fabp4"        
  [73] "Tnip3"         "Esam"          "Reg3g"         "Cenpf"        
  [77] "Apold1"        "Il6"           "Ms4a7"         "Hspa1a"       
  [81] "Ifit1"         "Rsad2"         "Aif1"          "Fcrls"        
  [85] "Scarb1"        "Alox5ap"       "Rasl11a"       "Ifit3"        
  [89] "Gclc"          "Slc7a11"       "A330069E16Rik" "Tagln"        
  [93] "Clu"           "Srxn1"         "Pi16"          "Fcgr3"        
  [97] "Tnfaip6"       "Gpr116"        "Gpihbp1"       "Laptm5"       
 [101] "C5ar1"         "Mmrn2"         "Napsa"         "Hist1h2ao"    
 [105] "Myh11"         "Agl"           "Tppp3"         "Eltd1"        
 [109] "Cfp"           "Upk3a"         "Mfap5"         "Ccr7"         
 [113] "Cox4i2"        "E030010A14Rik" "Lilrb4"        "Ppbp"         
 [117] "Rasd1"         "Csf1r"         "AA467197"      "Iigp1"        
 [121] "Emr1"          "Krt16"         "Dusp2"         "Cd93"         
 [125] "Flt1"          "Timp3"         "Ptx3"          "Lrg1"         
 [129] "Gja4"          "Ramp3"         "Sfpi1"         "Pln"          
 [133] "Upk2"          "Sncg"          "Mpeg1"         "Acer2"        
 [137] "Fgl2"          "Hmox1"         "Osm"           "Mrc1"         
 [141] "Robo4"         "Ly6k"          "Hsph1"         "Mef2c"        
 [145] "Samsn1"        "Isg15"         "Bcl2a1a"       "Gimap6"       
 [149] "Itgb7"         "C3ar1"         "Klf2"          "C130074G19Rik"
 [153] "Apoc1"         "Rac2"          "ORF63"         "Clec7a"       
 [157] "Ereg"          "2810417H13Rik" "Inmt"          "Ms4a6c"       
 [161] "Krt6a"         "Ifitm6"        "Ly86"          "Krt4"         
 [165] "Has1"          "Grrp1"         "Rgs16"         "Cxcl14"       
 [169] "Ptprb"         "Dpt"           "Aspn"          "Zfp935"       
 [173] "Ifi205"        "F830016B08Rik" "S100a4"        "Thrsp"        
 [177] "Tnfsf9"        "Ctgf"          "Csf2rb"        "Ly6c1"        
 [181] "Car3"          "Il10"          "Lum"           "Higd1b"       
 [185] "Hamp"          "Actg2"         "H2-Oa"         "Kdr"          
 [189] "Tnf"           "Des"           "Sdpr"          "AI607873"     
 [193] "Adm"           "Prss23"        "Ccr2"          "Pld4"         
 [197] "Upk1b"         "Cd53"          "Ccl11"         "S1pr1"        
 [201] "Nrip2"         "Rgs4"          "Cd34"          "Cyp1a1"       
 [205] "Postn"         "Pcdh17"        "Krt14"         "Cxcl9"        
 [209] "Cdca3"         "Rgs2"          "Podxl"         "Angptl7"      
 [213] "Rbp4"          "Cyp2f2"        "Cyr61"         "Krt20"        
 [217] "Sparcl1"       "Ccl12"         "Filip1l"       "Myl9"         
 [221] "Il1rn"         "Abcg2"         "Ube2c"         "Krtdap"       
 [225] "Nrp1"          "Mfa

In [10]:
srat@meta.data

,orig.ident,nCount_RNA,nFeature_RNA,channel,tissue,subtissue,mouse.sex,mouse.id,percent.ercc,percent.ribo,free_annotation,cell_ontology_class,res.0.4,previous_free_annotation,previous_cell_ontology_class,cluster.ids,cell_ontology_id
,<fct>,<dbl>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
10X_P4_3_AAAGTAGAGATGCCAG,10X,10291,2853,10X_P4_3,Bladder,,M,3-M-8,0,0.21960937,Bladder mesenchymal cell,bladder cell,0,NA,NA,0,CL:1001319
10X_P4_3_AACCGCGTCCAACCAA,10X,17520,4194,10X_P4_3,Bladder,,M,3-M-8,0,0.17671233,Bladder mesenchymal cell,bladder cell,2,NA,NA,2,CL:1001319
10X_P4_3_AACTCCCGTCGGGTCT,10X,12944,3347,10X_P4_3,Bladder,,M,3-M-8,0,0.19244438,Bladder mesenchymal cell,bladder cell,5,NA,NA,5,CL:1001319
10X_P4_3_AACTCTTAGTTGCAGG,10X,19594,3851,10X_P4_3,Bladder,,M,3-M-8,0,0.35638461,Luminal bladder epithelial cell,bladder urothelial cell,3,NA,NA,3,CL:1001428
10X_P4_3_AACTCTTTCATAACCG,10X,14594,3809,10X_P4_3,Bladder,,M,3-M-8,0,0.15711936,Bladder mesenchymal cell,bladder cell,2,NA,NA,2,CL:1001319
10X_P4_3_AAGACCTAGATCCGAG,10X,19190,3608,10X_P4_3,Bladder,,M,3-M-8,0,0.23778009,Monocyte/Macrophage,leukocyte,6,NA,NA,6,CL:0000738
10X_P4_3_AAGACCTAGGTGGGTT,10X,24484,4347,10X_P4_3,Bladder,,M,3-M-8,0,0.32809182,Basal bladder epithelial cell,bladder urothelial cell,1,NA,NA,1,CL:1001428
10X_P4_3_AAGGAGCGTGCAACTT,10X,5005,1960,10X_P4_3,Bladder,,M,3-M-8,0,0.19020979,Bladder mesenchymal cell,bladder cell,0,NA,NA,0,CL:1001319
10X_P4_3_AAGTCTGAGATAGTCA,10X,6253,2236,10X_P4_3,Bladder,,M,3-M-8,0,0.22597153,Luminal bladder epithelial cell,bladder urothelial cell,4,NA,NA,4,CL:1001428


In [12]:
srat[['RNA']]@meta.features

,vst.mean,vst.variance,vst.variance.expected,vst.variance.standardized,vst.variable
,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
Xkr4,0.0016,0.0015980792,0.0017496294,0.9133815,FALSE
Rp1,0.0004,0.0004000000,0.0004005521,0.9986216,FALSE
Sox17,0.0428,0.2546700280,0.0551756210,3.5306000,TRUE
Mrpl15,0.4000,0.4793917567,0.5880334304,0.8152458,FALSE
Lypla1,0.5512,0.7284699480,0.8741697112,0.8333278,FALSE
Tcea1,1.0472,1.5912086435,2.1291791708,0.7473343,FALSE
Rgs20,0.0008,0.0007996799,0.0008360070,0.9565468,FALSE
Atp6v1h,0.3992,0.5424563425,0.5865984707,0.9247490,FALSE
Oprk1,0.0060,0.0075670268,0.0072277315,1.0469435,FALSE


In [15]:
srat <- subset(srat, features=VariableFeatures(srat))
srat

An object of class Seurat 
2000 features across 2500 samples within 1 assay 
Active assay: RNA (2000 features, 2000 variable features)
 2 layers present: counts, data

In [4]:
info <- print(srat)
# RedisInfo(unique_id, info)

An object of class Seurat 
23341 features across 2500 samples within 1 assay 
Active assay: RNA (23341 features, 0 variable features)
 2 layers present: counts, data


In [6]:
assay_names <- names(srat@assays)
        
if(length(assay_names)==1){
    assay <- DefaultAssay(srat) # If there is only one assay, then no matter if assay is provided, set assay to default assay
} else if (assay %in% assay_names){
    DefaultAssay(srat) <- assay # If there is more than one assay, and the user provides assay, then set default assay to assay
} 
default_assay <- DefaultAssay(srat)
RedisInfo(unique_id, paste0("Setting default_assay to ", default_assay))

if(IsNormalized(srat[[default_assay]]@counts, min_genes=min_genes)){
    RedisError(unique_id, "Seurat QC only takes raw counts, not normalized data.")
    stop("Seurat QC only takes raw counts, not normalized data.")
}

[1] 2

In [ ]:
local({
   load("/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.Robj")
   ls()
})

In [ ]:
name <- load("/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.Robj")

In [ ]:
get(name)

In [ ]:
tiss

In [ ]:
typeof(tiss)

In [ ]:
tiss_new <- UpdateSeuratObject(tiss)
tiss_new

In [ ]:
tiss_new

In [ ]:
tiss_new[['RNA']]@counts

In [ ]:
tiss_new@meta.data

In [ ]:
tiss_new@var.features

In [ ]:
srat <- load("/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.Robj")
srat

In [ ]:
dim(srat)

In [ ]:
inupt_path <- "/ps/ai-ready/data/tung/tung.h5Seurat"

In [ ]:
srat <- LoadSeurat(inupt_path)
srat

In [ ]:
names(srat@assays)

In [ ]:
'RNA' %in% names(srat@assays)

In [ ]:
DefaultAssay(srat)<-'SCT'

In [ ]:
names(srat@assays)

In [ ]:
DefaultAssay(srat)

In [ ]:
get_assays_from_seurat <- function(seurat_object) {
    assays_names <- 'RNA'

    if (DefaultAssay(seurat_object) != 'RNA') {
        assays_names <- names(srat@assays)
    }
    assays_names
}

In [ ]:
assays_names <- get_assays_from_seurat(srat)
assays_names

In [ ]:
SaveH5Seurat(srat, filename = "/ps/ai-ready/data/tung/tung1.h5Seurat", overwrite = TRUE, verbose = FALSE)

In [ ]:
seu = DietSeurat(
  srat,
  counts = TRUE, # so, raw counts save to adata.layers['counts']
  data = TRUE, # so, log1p counts save to adata.X when scale.data = False, else adata.layers['data']
  scale.data = FALSE, # if only scaled highly variable gene, the export to h5ad would fail. set to false
  features = rownames(srat), # export all genes, not just top highly variable genes
  assays = "RNA",
  dimreducs = c("pca","umap"),
  graphs = c("RNA_nn", "RNA_snn"), # to RNA_nn -> distances, RNA_snn -> connectivities
  misc = TRUE
)

In [ ]:
seu[['RNA']]@counts

In [ ]:
seu[['RNA']]

In [ ]:
path <- MuDataSeurat::WriteH5AD(seu, "/ps/ai-ready/data/tung/seu.h5ad", assay="RNA")
path

In [ ]:
DefaultAssay(srat)<-'SCT'

seu_sct = DietSeurat(
  srat,
  counts = TRUE, # so, raw counts save to adata.layers['counts']
  data = TRUE, # so, log1p counts save to adata.X when scale.data = False, else adata.layers['data']
  scale.data = FALSE, # if only scaled highly variable gene, the export to h5ad would fail. set to false
  features = rownames(srat), # export all genes, not just top highly variable genes
  assays = "SCT",
  dimreducs = c("pca","umap"),
  graphs = c("RNA_nn", "RNA_snn"), # to RNA_nn -> distances, RNA_snn -> connectivities
  misc = TRUE
)
seu_sct

In [ ]:
seu_sct[['SCT']]@counts

In [ ]:
DefaultAssay(seu_sct)<-'RNA'

In [ ]:
path <- "/ps/ai-ready/data/tung/umi.h5Seurat"
seurat_object <- LoadH5Seurat(path)
seurat_object

In [ ]:
DefaultAssay(seurat_object)

In [ ]:
path <- "/ps/ai-ready/data/tung/umi.h5Seurat"
results <- convert_seurat_to_anndata(path, assay = 'RNA')

In [ ]:
# inupt_path <- "/ps/ai-ready/data/filtered_gene_bc_matrices/hg19/"
# inupt_path <- "/ps/ai-ready/data/kbcfh/Anndata/hca_heart_neuronal_raw.h5ad"
# path_of_scrublet_calls<-"/ps/ai-ready/data/filtered_gene_bc_matrices/hg19/scrublet_calls.tsv"
inupt_path = "/ps/ai-ready/data/error/droplet_Bladder_seurat_tiss.h5seurat"

In [ ]:
srat <- LoadSeurat(inupt_path)
srat

In [ ]:
default_assay <- "RNA"

In [ ]:
if(!default_assay %in% names(x = srat)) stop(paste(default_assay, "does not exist."))
DefaultAssay(object = srat) <-default_assay

In [ ]:
head(srat[[]])

In [ ]:
srat[[default_assay]]@counts

In [ ]:
if(!paste0("nCount_", default_assay) %in% names(x = srat[[]])) srat[[paste0("nCount_", default_assay)]] <- colSums(x = srat[[default_assay]], slot = "counts")  # nCount of the default assay
if(!paste0("nFeature_", default_assay) %in% names(x = srat[[]])) srat[[paste0("nFeature_", default_assay)]] <- colSums(x = GetAssayData(object = srat[[default_assay]], slot = "counts") > 0)  # nFeature of the default assay

In [ ]:
meta <- srat@meta.data
dim(meta)
head(meta)

summary(meta[paste0("nCount_", default_assay)])
summary(meta[paste0("nFeature_", default_assay)])

In [ ]:
srat[["percent.mt"]] <- PercentageFeatureSet(srat, pattern = "^MT-") # Michochondrial genes
srat[["percent.rb"]] <- PercentageFeatureSet(srat, pattern = "^RP[SL]") # Ribosomal proteins (their names begin with RPS or RPL

In [ ]:
if(! "percent.hb" %in% names(x = srat[[]])) srat[["percent.hb"]] <- PercentageFeatureSet(srat, pattern = "^HB[^(P)]")
if(! "percent.plat" %in% names(x = srat[[]])) srat[["percent.plat"]] <- PercentageFeatureSet(srat, pattern = "PECAM1|PF4")

In [ ]:
# add the doublet annotation
doublets <- read.table(path_of_scrublet_calls, header = F, row.names = 1)
colnames(doublets) <- c("Doublet_score", "Is_doublet")
srat <- AddMetaData(srat, doublets)
head(srat[[]])

In [ ]:
names(srat[[]])

In [ ]:
srat[[]]["scrublet_score"]

In [ ]:
doublets

In [ ]:
VlnPlot(srat, features = c(paste0("nFeature_", default_assay), paste0("nCount_", default_assay),"percent.mt","percent.rb"),ncol = 4,pt.size = 0.1) & 
  theme(plot.title = element_text(size=10))

In [ ]:
FeatureScatter(srat, feature1 = paste0("nCount_", default_assay), feature2 = "percent.mt")
FeatureScatter(srat, feature1 = paste0("nCount_", default_assay), feature2 = paste0("nFeature_", default_assay))
FeatureScatter(srat, feature1 = paste0("nCount_", default_assay), feature2 = "percent.rb")
FeatureScatter(srat, feature1 = "percent.rb", feature2 = "percent.mt")
# FeatureScatter(srat, feature1 = paste0("nFeature_", default_assay), feature2 = "Doublet_score")
FeatureScatter(srat, feature1 = paste0("nFeature_", default_assay), feature2 = "scrublet_score")

In [ ]:
srat[['QC']] <- ifelse(srat@meta.data$Is_doublet == 'True','Doublet','Pass')
srat[['QC']] <- ifelse(srat@meta.data[paste0("nFeature_", default_assay)] < 500 & srat@meta.data$QC == 'Pass','Low_nFeature',srat@meta.data$QC)
srat[['QC']] <- ifelse(srat@meta.data[paste0("nFeature_", default_assay)] < 500 & srat@meta.data$QC != 'Pass' & srat@meta.data$QC != 'Low_nFeature',paste('Low_nFeature',srat@meta.data$QC,sep = ','),srat@meta.data$QC)
srat[['QC']] <- ifelse(srat@meta.data$percent.mt > 15 & srat@meta.data$QC == 'Pass','High_MT',srat@meta.data$QC)
srat[['QC']] <- ifelse(srat@meta.data[paste0("nFeature_", default_assay)] < 500 & srat@meta.data$QC != 'Pass' & srat@meta.data$QC != 'High_MT',paste('High_MT',srat@meta.data$QC,sep = ','),srat@meta.data$QC)
table(srat[['QC']])

In [ ]:
srat[[]]

In [ ]:
VlnPlot(srat, features = c("nFeature_RNA","nCount_RNA","percent.mt","percent.rb"),ncol = 4,pt.size = 0.1) & 
  theme(plot.title = element_text(size=10))

In [ ]:
FeatureScatter(srat, feature1 = "nCount_RNA", feature2 = "percent.mt")
FeatureScatter(srat, feature1 = "nCount_RNA", feature2 = "nFeature_RNA")
FeatureScatter(srat, feature1 = "nCount_RNA", feature2 = "percent.rb")
FeatureScatter(srat, feature1 = "percent.rb", feature2 = "percent.mt")
FeatureScatter(srat, feature1 = "nFeature_RNA", feature2 = "Doublet_score")

In [ ]:
srat[['QC']] <- ifelse(srat@meta.data$Is_doublet == 'True','Doublet','Pass')
srat[['QC']] <- ifelse(srat@meta.data$nFeature_RNA < 500 & srat@meta.data$QC == 'Pass','Low_nFeature',srat@meta.data$QC)
srat[['QC']] <- ifelse(srat@meta.data$nFeature_RNA < 500 & srat@meta.data$QC != 'Pass' & srat@meta.data$QC != 'Low_nFeature',paste('Low_nFeature',srat@meta.data$QC,sep = ','),srat@meta.data$QC)
srat[['QC']] <- ifelse(srat@meta.data$percent.mt > 15 & srat@meta.data$QC == 'Pass','High_MT',srat@meta.data$QC)
srat[['QC']] <- ifelse(srat@meta.data$nFeature_RNA < 500 & srat@meta.data$QC != 'Pass' & srat@meta.data$QC != 'High_MT',paste('High_MT',srat@meta.data$QC,sep = ','),srat@meta.data$QC)
table(srat[['QC']])

In [ ]:
srat[['QC']]

In [ ]:
VlnPlot(subset(srat, subset = QC == 'Pass'), 
        features = c("nFeature_RNA", "nCount_RNA", "percent.mt","percent.rb"), ncol = 4, pt.size = 0.1) & 
  theme(plot.title = element_text(size=10))